In [1]:
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
import networkx as nx
import time
import matplotlib.animation as animation
%matplotlib qt

In [68]:
def nearest_neighbor(Edges, weights):
    """
        E: Arreglo de aristas del grafo
        d: distancias de cada arista
    """
    
    next_edge = np.argmin(weights)
    
    T = list(Edges[next_edge, :])
    ordered_el = [Edges[next_edge]]
    S = weights[next_edge]

    E = np.delete(Edges, next_edge, axis=0)
    d = np.delete(weights, next_edge, axis=0)


    while len(E) > 0:
        # Encontrar minima distancia conectada a un extremo del recorrido
        next_edge = np.argmin(np.where(np.any(np.isin(E, [T[0], T[-1]]), axis=1), d, np.inf))

        # Revisar si la arista va al principio o al final del recorrido
        a, b = E[next_edge]
        if a == T[0]: 
            T.insert(0, b)
        elif b == T[0]:
            T.insert(0, a)
            b, a = a, b
        elif a == T[-1]:
            T.append(b)
        else:
            T.append(a)
            b, a = a, b
        ordered_el.append(E[next_edge])
        S += d[next_edge]

        # a es nodo extremo previo, b es el nuevo extremo
        mask = ~np.logical_or(np.any(E == a, axis=1), np.all(np.isin(E, T), axis=1))
        E = E[mask]
        d = d[mask]

    a, b = np.sort([T[0], T[-1]])
    ordered_el.append([a, b])
    
    for edg, dist in zip(Edges, weights):
        if edg[0] == a and edg[1] == b:
            S += dist

    return T, S, ordered_el

In [69]:
n = 10
#C = np.loadtxt("data.txt", usecols=(1, 2), max_rows=n)
C = np.loadtxt("datos_unicos.txt", max_rows=n)
E = np.array(list(combinations(range(n), 2)))
d = np.linalg.norm(C[E[:, 0], :] - C[E[:, 1], :], axis=1)

In [70]:
# Guardar datos unicos
#np.savetxt("datos_unicos.txt", np.unique(C, axis=0))

In [71]:
t = time.perf_counter()
T, S, ordered_el = nearest_neighbor(E, d)
t = time.perf_counter() - t
t

0.0015293000001292967

In [72]:
S

710.3022594871238

In [73]:
G = nx.Graph()
G.add_weighted_edges_from(zip(E[:, 0], E[:, 1], d))

In [74]:
def edges_from_tour(T):
    return [(x, y) for x, y in zip(T, T[1:])] + [(T[-1], T[0])]

In [75]:
locs = dict(zip(range(n), C[:n, :]))
edgs = edges_from_tour(T)

nx.draw_networkx(G, locs, edgelist=edgs, node_size=200)
plt.show()

In [38]:
fig, ax = plt.subplots(figsize=(10, 10))
nx.draw_networkx(G, locs, edgelist=[], node_size=200, ax=ax)
def anim_tour(i):
    nx.draw_networkx_edges(G, locs, [ordered_el[i]])
#ax.set_aspect('equal')
anim = animation.FuncAnimation(fig, anim_tour, range(len(edgs)), interval=1000)
plt.show()

In [85]:
N = [5, 10, 20, 30, 40, 50, 75, 100, 125, 150, 200]
bench = np.zeros([len(N), 3])
for n, row in zip(N, bench):
    C = np.loadtxt("datos_unicos.txt", max_rows=n)
    E = np.array(list(combinations(range(n), 2)))
    d = np.linalg.norm(C[E[:, 0], :] - C[E[:, 1], :], axis=1)

    t = time.perf_counter()
    _, S, oredered_el = nearest_neighbor(E, d)
    t = time.perf_counter() - t
    row[:] =[n, t, S] 

np.savetxt("benchmark_NN.txt", bench)